<a href="https://colab.research.google.com/github/SantanaSDS/TGGuruji/blob/main/Guruji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install aiogram requests pandas lxml

In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
!apt-get update
!apt-get remove -y chromium-browser
!apt-get install -y chromium chromium-driver
!pip install --upgrade selenium


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
!apt-get update
!apt-get remove -y chromium-browser
!apt-get install -y chromium chromium-driver
!pip install --upgrade selenium


In [ ]:
pip install --upgrade aiogram


In [ ]:
import os
import json
import asyncio
import aiohttp
import nest_asyncio
import urllib.parse
import sys
import subprocess

# Пытаемся импортировать aiogram, если не найден — устанавливаем
try:
    from aiogram import Bot, Dispatcher, types
    from aiogram.filters import Command
    from aiogram.fsm.state import State, StatesGroup
    from aiogram.fsm.context import FSMContext
    from aiogram.fsm.storage.memory import MemoryStorage
    from aiogram.client.default import DefaultBotProperties
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "aiogram"])
    from aiogram import Bot, Dispatcher, types
    from aiogram.filters import Command
    from aiogram.fsm.state import State, StatesGroup
    from aiogram.fsm.context import FSMContext
    from aiogram.fsm.storage.memory import MemoryStorage
    from aiogram.client.default import DefaultBotProperties

# Загружаем ключи из переменных окружения
TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN", "7247766600:AAGCu9k0vcfDzECX0okvgCUhJxU0lOI7UvU")
PROXY_API_KEY = os.getenv("PROXY_API_KEY", "sk-n41nzmsVHJ1UC4B2KZy90xadrlPFGl6S")
BASE_URL = "https://santanaherbals.ru"

# Инициализация бота, диспетчера и хранилища состояний
bot = Bot(token=TELEGRAM_TOKEN, default=DefaultBotProperties(parse_mode="HTML"))
storage = MemoryStorage()
dp = Dispatcher(storage=storage)

# Группа состояний для активации разговора
class ConversationStates(StatesGroup):
    active = State()

# Роль бота с правилами ответа для GPT (не выводится пользователю)
BOT_ROLE = (
    "Ты - аюрведический консультант. Отвечай кратко и понятно, используя принципы Аюрведы.\n\n"
    "**Правила ответа:**\n"
    "1. Если пользователь спрашивает о заболевании, состоянии или ищет травы для лечения (например, \"что помогает от...\", \"аюрведа при...\", \"лучшие травы для...\"):\n"
    "   - Дай объяснение на 600-800 символов причины дисбаланса в терминах дош (Вата, Питта, Капха).\n"
    "   - Опиши, какие доши выходят из равновесия и их симптомы.\n"
    "   - В конце ответа выдай JSON-массив из 4–6 подходящих аюрведических трав на русском языке без пояснений.\n"
    "   - Пример ответа:\n"
    "     В Аюрведе бессонница часто связана с дисбалансом Вата-доши, которая отвечает за нервную систему. При избытке Ваты человек может испытывать тревожность, сухость кожи, напряжение в теле и поверхностный сон. Для восстановления баланса рекомендуется тёплая, питательная пища, регулярный режим сна и натуральные травы.\n"
    "     [\"брахми\", \"ашвагандха\", \"шатавари\", \"мускатный орех\"]\n\n"
    "2. Если пользователь спрашивает о конкретной траве или продукте (например, \"От чего ашваганда?\", \"Для чего трифала?\"):\n"
    "   - Дай описание растения на 600-800 символов, его влияние на доши и основные полезные свойства.\n"
    "   - Не включай список других трав.\n"
    "   - В JSON-массиве укажи только запрашиваемое растение.\n"
    "   - Пример ответа:\n"
    "     Ашваганда, также известная как индийский женьшень, считается одной из лучших адаптогенных трав в Аюрведе. Она балансирует Вата и Капха доши, помогая организму справляться со стрессом, улучшает сон и повышает уровень энергии.\n"
    "     [\"ашвагандха\"]\n\n"
    "**Дополнительные правила:**\n"
    "- Если запрос касается конкретного растения, JSON должен содержать только это растение.\n"
    "- Если запрос общий (о заболевании, симптомах, состоянии), JSON должен содержать список из 4–6 трав.\n"
    "- Не давай медицинских диагнозов, а только объясняй состояния с точки зрения Аюрведы.\n"
    "Всегда следуй этим правилам и не добавляй ничего лишнего."
    "Рекомендуй обратиться за полноценной консультацией и пройти полную диагностику"
)

# Функция для запроса рекомендаций через GPT
async def gpt_suggest_herbs(query: str) -> str:
    url = "https://api.proxyapi.ru/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {PROXY_API_KEY}",
        "Content-Type": "application/json"
    }
    # Передаем правила вместе с запросом
    prompt = f"""
{BOT_ROLE}
Клиент спрашивает: "{query}"
"""
    data = {
        "model": "gpt-4o",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }
    try:
        async with aiohttp.ClientSession() as session:
            async with session.post(url, json=data, headers=headers) as response:
                if response.status == 200:
                    result = await response.json()
                    return result["choices"][0]["message"]["content"]
                else:
                    return "❌ Ошибка в обработке запроса. Попробуйте позже."
    except Exception as e:
        return f"❌ Ошибка: {str(e)}"

# Функция для извлечения JSON-массива трав из текста ответа
def extract_herbs_json(text: str) -> (str, list):
    text = text.replace("```json", "").replace("```", "")
    start_index = text.find('[')
    end_index = text.find(']', start_index)
    if start_index == -1 or end_index == -1:
        return (text, [])
    description = text[:start_index].strip()
    json_str = text[start_index:end_index + 1]
    try:
        herbs_list = json.loads(json_str)
    except json.JSONDecodeError:
        herbs_list = []
    return (description, herbs_list)

# Обработчик активации: бот активируется только после сообщения "Гуруджи"
@dp.message(lambda message: message.text.strip().lower() == "гуруджи")
async def activate_conversation(message: types.Message, state: FSMContext):
    await state.set_state(ConversationStates.active)
    # Приветствие без вывода правил
    await message.answer("Привет! Аюрведический консультант к вашим услугам. Чем могу помочь?")

# Обработчик запроса: срабатывает только если чат активирован
@dp.message()
async def handle_herb_query(message: types.Message, state: FSMContext):
    current_state = await state.get_state()
    if current_state != ConversationStates.active.state:
        return
    query = message.text.strip()
    expert_response = await gpt_suggest_herbs(query)
    description, herbs_list = extract_herbs_json(expert_response)

    formatted_text = f"<b>Описание:</b>\n{description}\n\n"
    if herbs_list:
        formatted_text += "<b>Подходящие травы:</b>\n"
        for herb in herbs_list:
            search_link = f'<a href="{BASE_URL}/catalog/?q={urllib.parse.quote(herb)}">Найти на сайте</a>'
            formatted_text += f"• <b>{herb}</b> — {search_link}\n"
    else:
        formatted_text += "Не удалось извлечь список трав из ответа."

    await message.answer(formatted_text, disable_web_page_preview=True)

# Функция для запуска бота
async def main():
    await dp.start_polling(bot)

if __name__ == '__main__':
    nest_asyncio.apply()
    asyncio.run(main())
